### **Transfer Learning for NLP:**
- Hotel reviews Dataset Loading
- Preprocessing the data
- Emlo embedding layer Class
- Model creation
- Training the model
- Model prediction

In [ ]:
! pip install nltk

In [ ]:
# Keras ELMO implementation requires older version of Tensorflow. Make sure to run this and restart your kernal. 
# Do not run this on local jupyter notebook. Use Google Colab or Cloud Notebooks with GPU's 
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15

     |████████████████████████████████| 412.3 MB 25 kB/s 
     |████████████████████████████████| 503 kB 36.7 MB/s 
     |████████████████████████████████| 3.8 MB 34.8 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=3ba885d4306831f3fdda4d177d785fe9f9fc27e571382da55c216498829b14b0
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd

Google drive mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Loading Hotel reviews Dataset:**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/tripadvisor_hotel_reviews.csv')
data.head()

Review  Rating
0  nice hotel expensive parking got good deal sta...       4
1  ok nothing special charge diamond member hilto...       2
2  nice rooms not 4* experience hotel monaco seat...       3
3  unique, great stay, wonderful time hotel monac...       5
4  great stay great stay, went seahawk game aweso...       5

### **Preprocessing the data:**

In [ ]:
data_subset = data[(data['Rating']==5) | (data['Rating']==1)]
data_subset = data_subset[['Review','Rating']]
data_subset.shape
data_subset.head()

Review  Rating
3  unique, great stay, wonderful time hotel monac...       5
4  great stay great stay, went seahawk game aweso...       5
5  love monaco staff husband stayed hotel crazy w...       5
6  cozy stay rainy city, husband spent 7 nights m...       5
8  hotel stayed hotel monaco cruise, rooms genero...       5

In [ ]:
data_sample = data_subset.sample(frac=0.02)
data_sample.shape

(210, 2)

Converting reviews to classes --> 1 or 0

In [ ]:
def convert_to_string(rating):
  if rating == 5.0:
    return 1
  else:
    return 0
  
data_sample['label'] = data_sample['Rating'].apply(convert_to_string)
data_sample.head()

Review  Rating  label
15228  great honeymoon quite simply greatest hotel st...       5      1
19185  renew soon hope travelling hotel got worried t...       5      1
4463   expected visit paris better imagined decision ...       5      1
1664   superb location nice clean rooms friend stayed...       5      1
666    fun weekend adams mark n't know definitely sta...       5      1

### **Emlo embedding layer Class:**

In [ ]:
class ElmoEmbeddingLayer(tf.keras.layers.Layer):
    """Original Author Credit --- Taken from: 
    https://github.com/strongio/keras-elmo/blob/master/Elmo%20Keras.ipynb"""
    def __init__(self, **kwargs):
        # the ELMO dimensions are set at 1024, which means the data fed to it, will be projected onto 1024 dimensions. 
        self.dimensions = 1024
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Downloading the ELMO Version 2 from TF HUB. 
        # We are setting trainable parameter to be True, since we will train our data with that. 
        self.elmo = hub.Module(
            'https://tfhub.dev/google/elmo/2', 
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )
        if self.trainable:
          # Trainable True will automatically set empty placeholders for weights used during training. 
          self._trainable_weights.extend(
              tf.trainable_variables(scope="^{}_module/.*".format(self.name))
          )
        # Changed assuming trainable weights might be set using 
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
      #Keras squeeze Removes dimensions of size 1 from the shape of a tensor. We are squeezing our input and cast them as strings 
      # while initializing our ELMO layer. 
        result = self.elmo(
            K.squeeze(K.cast(x, tf.string), axis=1),
            as_dict=True,
            signature='default',
        )['default']
        return result

    def compute_mask(self, inputs, mask=None):
      # adding padding to non uniform inputs before sending to model. 
      #this is geenral pre-processing step. 
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

### **Model creation:**

In [ ]:
X = np.array(data_sample['Review'].tolist()).reshape(data_sample.shape[0], 1)
y = np.array(data_sample['label'].tolist()).reshape(data_sample.shape[0], 1)
X.shape, y.shape

((210, 1), (210, 1))

In [ ]:
def create_model(train_elmo=False):
  # Create Sequential model
  model = tf.keras.Sequential([
      # Need to explicitly include input layer 
      # to allow keras to accept string input
      # Taken from:
      # https://gist.github.com/colinmorris/9183206284b4fe3179809098e809d009
      tf.keras.layers.InputLayer(dtype='string', input_shape=(1,)),
      ElmoEmbeddingLayer(trainable=train_elmo),
      tf.keras.layers.Dense(1)
  ])
  
  # Needed to initialize elmo variables
  sess = K.get_session()
  init = tf.global_variables_initializer()
  sess.run(init)
  
  # Compile model
  model.compile(
      optimizer="adam", 
      loss="binary_crossentropy", 
      metrics=["accuracy"]
  )
  return model

### **Reshaping data to adjust according to ELMO:**

In [ ]:
model = create_model(train_elmo=True)
# Building Model Object 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
model.fit(X, y,batch_size=16,epochs=1) #Elmo requires a very HIGH GPU configuration (not available on google colab)
# running on cpu will take lot of time, roughly 1 hour for single epoch. 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 210 samples
210/210 [==============================] - 40s 190ms/sample - loss: 2.3845 - acc: 0.6476


In [ ]:
model.summary() # model summary 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
elmo_embedding_layer (ElmoEm (None, 1024)              4         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 1,029
Trainable params: 1,029
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.trainable_weights # These are the weights which are re-trained using our data. These weights can later be used to productionize this model. 

[<tf.Variable 'elmo_embedding_layer_module/aggregation/weights:0' shape=(3,) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/aggregation/scaling:0' shape=() dtype=float32>,
 <tf.Variable 'dense/kernel:0' shape=(1024, 1) dtype=float32>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32>]

In [ ]:
elmo = model.layers[0].elmo

In [ ]:
elmo.variables # These are internal ELMO Layers. 
# You can see the power of transfer learning here. ELMO is already a trained model. You can see the layers used during initial training. 
# Once trained, it learns the basic pattern of English language which gets transfered to different data. 
# In our Data, we are using this trained model to learn positive and negative review. 
# This helps not focus on building your own deep learning model layers, but instead, use trained ELMO as plug and play. 

[<tf.Variable 'elmo_embedding_layer_module/aggregation/scaling:0' shape=() dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/aggregation/weights:0' shape=(3,) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/bilm/CNN/W_cnn_0:0' shape=(1, 1, 16, 32) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/bilm/CNN/W_cnn_1:0' shape=(1, 2, 16, 32) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/bilm/CNN/W_cnn_2:0' shape=(1, 3, 16, 64) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/bilm/CNN/W_cnn_3:0' shape=(1, 4, 16, 128) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/bilm/CNN/W_cnn_4:0' shape=(1, 5, 16, 256) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/bilm/CNN/W_cnn_5:0' shape=(1, 6, 16, 512) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/bilm/CNN/W_cnn_6:0' shape=(1, 7, 16, 1024) dtype=float32>,
 <tf.Variable 'elmo_embedding_layer_module/bilm/CNN/b_cnn_0:0' shape=(32,) dtype=float32>,
 <tf.Variable 'elmo_

In [ ]:
model.predict([["This product is not as per expectation"]]) # you can use model.predict to get the sentiment rating of reviews. 
# however the results are not promising because we only trained for one epoch. You can train it for more epochs and check the results. 

array([[-0.1703878]], dtype=float32)

In [ ]:
model.predict([["This is awesome"]])

array([[-0.38632026]], dtype=float32)